### Gumbel softmax trick

In [1]:
import numpy as np
import tensorflow as tf
from keras.initializers import RandomUniform
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Reshape, LSTM, Lambda, BatchNormalization, GaussianNoise, Flatten
from keras import backend as K
from keras.optimizers import Adam 
from keras.activations import softmax


def GumbelNoise(logits):
    """ Adds gumbels noise to the logits
        I generate the gumbel noise by 
        applying the inverse CDF to uniform
        noise. 
        
        The inverse CDF of the gumbel is
        -log( -log(x) )
        
    """
    U = K.random_uniform(K.shape(logits), 0, 1)
    y = logits - K.log(-K.log(U + 1e-20) + 1e-20) # logits + gumbel noise
    return y

input_dim = 2
output_dim = 2

gumbel_temperature = 0.1

        
#Usual  2-layer MLP with parameter noise
inp = Input(shape = (input_dim,))
x = Dense(256, activation='relu')(inp)
x = GaussianNoise(1.0)(x)
x = Dense(128, activation='relu')(x)
x = GaussianNoise(1.0)(x)
logits = Dense(output_dim)(x)

# Now do the softmax gumbel trick: (which outputs a one-hot vector)
# Apply softmax to (g_i + logits) / temperate
# where g_i is gumbel noise, and temperature is a 
# softness par (when small, almost exactly a one-hot vec)
z = Lambda(GumbelNoise)(logits)    #add noise
z = Lambda(lambda x: x / gumbel_temperature)(z) #divide by temperature
out = Dense(output_dim, activation='softmax')(z)  #then softmax
model =  Model(inp, out)

/home/kokeeffe/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [15]:
x = np.array([[1,1]])
a = model.predict(x)[0]
a

array([3.6366418e-05, 9.9996364e-01], dtype=float32)

In [19]:
np.argmax(a)
np.array([1 if i == np.argmax(a) else 0 for i in range(len(a))])

array([0, 1])